<a href="https://colab.research.google.com/github/HayyanWasi/openai-agent-learning/blob/main/07_HANDOFFS_AGENTSDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install openai google-generativeai openai-agents python-dotenv


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [31]:
from agents import Agent, Runner, AsyncOpenAI, set_default_openai_client, set_tracing_disabled, OpenAIChatCompletionsModel, RunConfig
import os


from google.colab import userdata


gemini_api_key = userdata.get('GEMINI_API_KEY')
set_tracing_disabled(True)

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/",
)


set_default_openai_client(external_client)


model = OpenAIChatCompletionsModel(
    model="models/gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
    )


In [33]:
from pydantic import BaseModel

class Tutorial(BaseModel):
  outline:str
  tutorial:str



#model which is used by the agent to generate tutorial
tutorial_genrator = Agent(
    name="Tutorial Generator",
    handoffs_description = "this is used for making tutorials based on outline builder", #use for precision in decision making
    instructions="your are You are TutorialGPT, an expert in generating clear, structured, and engaging tutorials on any topic. Your job is to create high-quality step-by-step tutorials tailored to the target audience’s skill level (beginner, intermediate, or advanced).",
    model=model,
    output_type=Tutorial
)
#used to generate outline after completing its work it can handoff its control to tutorial_generator to make tutorial around the outline after giving handoff an agent can not be return bacl
outline_builder= Agent(
    name="outline border",
    instructions=("You are OutlineGPT, an expert assistant that creates well-structured, logical outlines for programming tutorials."
    "Your task is to generate detailed tutorial outlines on any programming topic (language, framework, tool, or concept) based on the user’s input."),
    handoffs=[tutorial_genrator], #using handoff
    model=model
)
tutorial_response = await Runner.run(outline_builder,"i want to learn  loops in java")
print(tutorial_response.final_output)

outline='I. Introduction to Loops in Java\n   A. What are loops and why are they important?\n   B. Types of loops in Java: `for`, `while`, and `do-while`\n   C. Basic syntax and structure of each loop type\n\nII. The `for` Loop\n   A. Syntax and components of the `for` loop (initialization, condition, increment/decrement)\n   B. Examples of using `for` loops to iterate a specific number of times\n   C. Using `for` loops with arrays and collections\n   D. Nested `for` loops\n   E. Common mistakes and how to avoid them\n\nIII. The `while` Loop\n   A. Syntax and components of the `while` loop (condition)\n   B. Examples of using `while` loops when the number of iterations is unknown\n   C. Using `while` loops with boolean flags\n   D. Common mistakes and how to avoid them (infinite loops)\n\nIV. The `do-while` Loop\n   A. Syntax and components of the `do-while` loop (condition)\n   B. Key difference between `while` and `do-while` loops (guaranteed execution at least once)\n   C. Examples 

In [ ]:
# from agents import function_tool,handoff
from agents import function_tool,Agent, Runner, handoff, AssistantAgent
from pydantic import BaseModel

# Step 1: Define data model for handoff to manager
class ManagerEscalation(BaseModel):
    issue: str  # Issue being escalated
    why: str    # Why it can't be handled by current agent

# Step 2: Create a function tool to simulate ticket creation
@function_tool
def create_ticket(issue: str):
    """
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In production, integrate with an actual ticketing system

# Step 3: Define Manager agent
manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial customer service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

# Step 4: Define what happens during handoff
def on_manager_handoff(ctx, input: ManagerEscalation):
    print("Escalating to manager agent:", input.issue)
    print("Reason for escalation:", input.why)
    # Optional: log to a DB or monitoring tool here

# Step 5: Define Customer Service agent
customer_service_agent = Agent(
    name="Customer Service",
    instructions=(
        "You assist customers with general inquiries and basic troubleshooting. "
        "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue yourself."
    ),
    handoffs=[
        handoff(
            agent=manager_agent,
            input_type=ManagerEscalation,
            on_handoff=on_manager_handoff,
        )
    ],
)

# Step 6: Define the root AssistantAgent and include other agents
assistant = AssistantAgent(
    name="Support Assistant",
    instructions="You route customer issues to the appropriate agent for resolution.",
    agents=[customer_service_agent, manager_agent],
)

# Step 7: Run the assistant with a customer query
result = await Runner.run(assistant,
    "I want a refund, but your system won't let me process it. The website is just blank for me.")

# Step 8: Show the final result
print(result.final_output)
